# All imports and installs


In [1]:
!pip freeze > requirements.txt
# !pip install -r requirements.txt

In [2]:
# import libraries
import findspark
findspark.init()
from pyspark.sql.types import *
import pyspark.sql.functions as f
from pyspark.sql import SparkSession, Window, Row


## Spark initialize


In [3]:
spark = SparkSession.builder.appName(
    'Practise_for_certification').getOrCreate()

# local[*] # means use all the cores available on the local machine

your 131072x1 screen size is bogus. expect trouble
23/06/11 18:35:40 WARN Utils: Your hostname, AMRIT resolves to a loopback address: 127.0.1.1; using 172.17.145.161 instead (on interface eth0)
23/06/11 18:35:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/11 18:35:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Topics


## Dataframe


In [4]:
#  create df from csv 
df = spark.read.csv('Final_Rainfall.csv', header=True, inferSchema=True)
df.show(n=2, truncate=False)

+--------+-----------+-----------+--------------+-----+
|coord_id|lon        |lat        |date          |value|
+--------+-----------+-----------+--------------+-----+
|26      |85.92405469|27.19137527|6/10/2011 0:00|0.0  |
|26      |85.92405469|27.19137527|6/10/2011 0:30|0.0  |
+--------+-----------+-----------+--------------+-----+
only showing top 2 rows



In [5]:
df.count()

636240

In [6]:
country = spark.createDataFrame(
    [("USA", "New York"),
        ("India", "New Delhi"),
        ("China", "Beijing"),
        ("UK", "London")
     ],
    ["country", "capital"]
)
country.show(n=2, truncate=False)
country.printSchema()

+-------+---------+
|country|capital  |
+-------+---------+
|USA    |New York |
|India  |New Delhi|
+-------+---------+
only showing top 2 rows

root
 |-- country: string (nullable = true)
 |-- capital: string (nullable = true)



In [7]:
# recerate  using data types too
data = [("USA", "New York", 5),
        ("India", "New Delhi", 10),
        ("China", "Beijing", 15),
        ("UK", "London", 20),
        ("Nepal", "Kathmandu",None ),
        (None ,"islamabad" ,None ),
        ("Bhutan", None,30 )
        ]
schema = StructType([
    StructField("country", StringType(), True),
    StructField("capital", StringType(), True),
    StructField("count", IntegerType(), True)
])
country_count = spark.createDataFrame(data, schema)
country_count.show(n=2, truncate=False)
country_count.printSchema()

+-------+---------+-----+
|country|capital  |count|
+-------+---------+-----+
|USA    |New York |5    |
|India  |New Delhi|10   |
+-------+---------+-----+
only showing top 2 rows

root
 |-- country: string (nullable = true)
 |-- capital: string (nullable = true)
 |-- count: integer (nullable = true)



## Renaming columns and its type (cast)

In [8]:
#Rename count to Rank
country_rank=country_count.withColumnRenamed("count","Rank")
#syntax: withColumnRenamed(existingColName, newColName) 
country_rank.show()

+-------+---------+----+
|country|  capital|Rank|
+-------+---------+----+
|    USA| New York|   5|
|  India|New Delhi|  10|
|  China|  Beijing|  15|
|     UK|   London|  20|
|  Nepal|Kathmandu|null|
|   null|islamabad|null|
| Bhutan|     null|  30|
+-------+---------+----+



 withColumn("newColName ",df["existingColName"].cast(StringType()))

In [9]:
# cast Rank to string
country_rank=country_rank.withColumn("Rank",country_rank["Rank"].cast(StringType()))
#syntax: withColumn("newColName ",df["existingColName"].cast(StringType()))
country_rank.printSchema()


root
 |-- country: string (nullable = true)
 |-- capital: string (nullable = true)
 |-- Rank: string (nullable = true)



## Selecting columns

In [10]:
# country_rank=country_rank.select("country","Rank")
country_rank.select(f.col("country"),f.col("Rank")).show()

+-------+----+
|country|Rank|
+-------+----+
|    USA|   5|
|  India|  10|
|  China|  15|
|     UK|  20|
|  Nepal|null|
|   null|null|
| Bhutan|  30|
+-------+----+



In [11]:
# only cast selected solumns and  create seperate df
country_rank=country_count.selectExpr("country","cast(count as string) as Rank")
country_rank.printSchema()

root
 |-- country: string (nullable = true)
 |-- Rank: string (nullable = true)



In [12]:
# list columns
# country_rank.columns
country_count.columns

['country', 'capital', 'count']

In [13]:
 country_count.select("country", "capital").show()   
country_count.drop("count").show()


+-------+---------+
|country|  capital|
+-------+---------+
|    USA| New York|
|  India|New Delhi|
|  China|  Beijing|
|     UK|   London|
|  Nepal|Kathmandu|
|   null|islamabad|
| Bhutan|     null|
+-------+---------+

+-------+---------+
|country|  capital|
+-------+---------+
|    USA| New York|
|  India|New Delhi|
|  China|  Beijing|
|     UK|   London|
|  Nepal|Kathmandu|
|   null|islamabad|
| Bhutan|     null|
+-------+---------+



## Filtering

In [14]:
country_count.filter(country_count["count"] > 10).show()
country_count.where(country_count["count"] > 10).show()

+-------+-------+-----+
|country|capital|count|
+-------+-------+-----+
|  China|Beijing|   15|
|     UK| London|   20|
| Bhutan|   null|   30|
+-------+-------+-----+

+-------+-------+-----+
|country|capital|count|
+-------+-------+-----+
|  China|Beijing|   15|
|     UK| London|   20|
| Bhutan|   null|   30|
+-------+-------+-----+



## Dropping columns

In [15]:
country_count.drop("capital").show()

cols_to_drop = ("capital", "count")
country_count.drop(*cols_to_drop).show()

+-------+-----+
|country|count|
+-------+-----+
|    USA|    5|
|  India|   10|
|  China|   15|
|     UK|   20|
|  Nepal| null|
|   null| null|
| Bhutan|   30|
+-------+-----+

+-------+
|country|
+-------+
|    USA|
|  India|
|  China|
|     UK|
|  Nepal|
|   null|
| Bhutan|
+-------+



In [16]:
# drop null values
country_count.dropna().show()  # silimiar to "any"
#any  
country_count.dropna(how="any").show()  # default

#all
country_count.dropna(how="all").show() # drop if all values are null in a row

#  drop null values in specific columns 
country_count.dropna(subset=["capital"]).show()



+-------+---------+-----+
|country|  capital|count|
+-------+---------+-----+
|    USA| New York|    5|
|  India|New Delhi|   10|
|  China|  Beijing|   15|
|     UK|   London|   20|
+-------+---------+-----+

+-------+---------+-----+
|country|  capital|count|
+-------+---------+-----+
|    USA| New York|    5|
|  India|New Delhi|   10|
|  China|  Beijing|   15|
|     UK|   London|   20|
+-------+---------+-----+

+-------+---------+-----+
|country|  capital|count|
+-------+---------+-----+
|    USA| New York|    5|
|  India|New Delhi|   10|
|  China|  Beijing|   15|
|     UK|   London|   20|
|  Nepal|Kathmandu| null|
|   null|islamabad| null|
| Bhutan|     null|   30|
+-------+---------+-----+

+-------+---------+-----+
|country|  capital|count|
+-------+---------+-----+
|    USA| New York|    5|
|  India|New Delhi|   10|
|  China|  Beijing|   15|
|     UK|   London|   20|
|  Nepal|Kathmandu| null|
|   null|islamabad| null|
+-------+---------+-----+



##  Union
add new  rows to the dataframe

In [17]:
df1=spark.createDataFrame([("A",1),("B",2),("C",3),("D",4)],["TEXT","NUMBER"])

df2=spark.createDataFrame([("C",3),("D",4),("E",5),("F",6)],["TEXT","NUMBER"])
df1.union(df2).show() # also show duplicates
df1.unionAll(df2).show() # also show duplicates

df1.union(df2).distinct().show() # remove duplicates

+----+------+
|TEXT|NUMBER|
+----+------+
|   A|     1|
|   B|     2|
|   C|     3|
|   D|     4|
|   C|     3|
|   D|     4|
|   E|     5|
|   F|     6|
+----+------+

+----+------+
|TEXT|NUMBER|
+----+------+
|   A|     1|
|   B|     2|
|   C|     3|
|   D|     4|
|   C|     3|
|   D|     4|
|   E|     5|
|   F|     6|
+----+------+

+----+------+
|TEXT|NUMBER|
+----+------+
|   A|     1|
|   B|     2|
|   C|     3|
|   D|     4|
|   E|     5|
|   F|     6|
+----+------+



## Sorting

In [30]:
data = [("Alice", "Science", 80),("Alice", "Math", 90), ("Alice", "English", 85), ("Mark", "Math", 85),("Mark", "English", 80),("Penny", "Science", 85),("Penny", "Math", 90),("Penny", "English", 75),("Gwen", "Science", 75),("Gwen", "Math", 95),("Gwen", "English", 80)]
schema = ["student_name", "subject", "marks"]
marks_df = spark.createDataFrame(data, schema)
marks_df.show()

+------------+-------+-----+
|student_name|subject|marks|
+------------+-------+-----+
|       Alice|Science|   80|
|       Alice|   Math|   90|
|       Alice|English|   85|
|        Mark|   Math|   85|
|        Mark|English|   80|
|       Penny|Science|   85|
|       Penny|   Math|   90|
|       Penny|English|   75|
|        Gwen|Science|   75|
|        Gwen|   Math|   95|
|        Gwen|English|   80|
+------------+-------+-----+



In [31]:
marks_df.sort("student_name").show(4)
marks_df.sort("student_name", ascending=False).show(4)   # ascending=False means descending order
marks_df.sort(f.col("student_name").desc()).show(4)  # .desc() is same as ascending=False
marks_df.sort("student_name","marks").show(4)

+------------+-------+-----+
|student_name|subject|marks|
+------------+-------+-----+
|       Alice|   Math|   90|
|       Alice|Science|   80|
|       Alice|English|   85|
|        Gwen|Science|   75|
+------------+-------+-----+
only showing top 4 rows

+------------+-------+-----+
|student_name|subject|marks|
+------------+-------+-----+
|       Penny|Science|   85|
|       Penny|English|   75|
|       Penny|   Math|   90|
|        Mark|   Math|   85|
+------------+-------+-----+
only showing top 4 rows

+------------+-------+-----+
|student_name|subject|marks|
+------------+-------+-----+
|       Penny|Science|   85|
|       Penny|English|   75|
|       Penny|   Math|   90|
|        Mark|   Math|   85|
+------------+-------+-----+
only showing top 4 rows

+------------+-------+-----+
|student_name|subject|marks|
+------------+-------+-----+
|       Alice|Science|   80|
|       Alice|English|   85|
|       Alice|   Math|   90|
|        Gwen|Science|   75|
+------------+-------+----

## Convert df to TempView
to run sql queries on df

In [32]:
marks_df.createTempView("marks_table") 

AnalysisException: [TEMP_TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create the temporary view `marks_table` because it already exists.
Choose a different name, drop or replace the existing view,  or add the IF NOT EXISTS clause to tolerate pre-existing views.

In [36]:
spark.sql("select * from marks_table order by marks DESC").show(4)

+------------+-------+-----+
|student_name|subject|marks|
+------------+-------+-----+
|        Gwen|   Math|   95|
|        Mark|Science|   90|
|       Penny|   Math|   90|
|       Alice|   Math|   90|
+------------+-------+-----+
only showing top 4 rows



## Basic Aggregration

In [37]:
marks_df.groupBy("student_name").agg(f.min("marks")).show() # min marks of each student
marks_df.groupBy("student_name").agg(f.sum("marks")).show() # sum of marks of each student
marks_df.groupBy("student_name").agg(f.avg("marks")).show() # avg of marks of each student

# similary other aggregration functions can be used like count , max , min , sum , avg , mean , stddev , variance 

+------------+----------+
|student_name|min(marks)|
+------------+----------+
|       Alice|        80|
|        Mark|        80|
|       Penny|        75|
|        Gwen|        75|
+------------+----------+

+------------+----------+
|student_name|sum(marks)|
+------------+----------+
|       Alice|       255|
|        Mark|       165|
|       Penny|       250|
|        Gwen|       250|
+------------+----------+

+------------+-----------------+
|student_name|       avg(marks)|
+------------+-----------------+
|       Alice|             85.0|
|        Mark|             82.5|
|       Penny|83.33333333333333|
|        Gwen|83.33333333333333|
+------------+-----------------+



## Joins

use keys to joins df

In [38]:
science_df=marks_df.filter(marks_df["subject"]=="Science")
math_df=marks_df.filter(marks_df["subject"]=="Math")

science_df.show()
math_df.show()

+------------+-------+-----+
|student_name|subject|marks|
+------------+-------+-----+
|       Alice|Science|   80|
|       Penny|Science|   85|
|        Gwen|Science|   75|
+------------+-------+-----+

+------------+-------+-----+
|student_name|subject|marks|
+------------+-------+-----+
|       Alice|   Math|   90|
|        Mark|   Math|   85|
|       Penny|   Math|   90|
|        Gwen|   Math|   95|
+------------+-------+-----+



In [45]:
#syntax: df1.join(df2,df1["col1"]==df2["col2"],<join_type>)
science_df.join(math_df,science_df["student_name"]==math_df["student_name"],"inner").show() # inner join
science_df.join(math_df,["student_name"],"left").show() # left join   # can use single key column name if both df has same column name , so no duplicate column name
science_df.join(math_df,science_df["student_name"]==math_df["student_name"],"right").show() # right join
science_df.join(math_df,science_df["student_name"]==math_df["student_name"],"full").show() # full join or outer join so list all rows from both df

#left_semi join
science_df.join(math_df,science_df["student_name"]==math_df["student_name"],"left_semi").show() # left_semi join

#left_anti join
science_df.join(math_df,science_df["student_name"]==math_df["student_name"],"left_anti").show() # left_anti join # list all rows from left df which are not in right df

+------------+-------+-----+------------+-------+-----+
|student_name|subject|marks|student_name|subject|marks|
+------------+-------+-----+------------+-------+-----+
|       Alice|Science|   80|       Alice|   Math|   90|
|        Gwen|Science|   75|        Gwen|   Math|   95|
|       Penny|Science|   85|       Penny|   Math|   90|
+------------+-------+-----+------------+-------+-----+

+------------+-------+-----+-------+-----+
|student_name|subject|marks|subject|marks|
+------------+-------+-----+-------+-----+
|       Alice|Science|   80|   Math|   90|
|       Penny|Science|   85|   Math|   90|
|        Gwen|Science|   75|   Math|   95|
+------------+-------+-----+-------+-----+

+------------+-------+-----+------------+-------+-----+
|student_name|subject|marks|student_name|subject|marks|
+------------+-------+-----+------------+-------+-----+
|       Alice|Science|   80|       Alice|   Math|   90|
|        null|   null| null|        Mark|   Math|   85|
|       Penny|Science|   

## Partition (Colesce and repartition)

In [49]:
# Read Final_Rainfall.csv file
df_RailFall = spark.read.csv('Final_Rainfall.csv', header=True, inferSchema=True)
df_RailFall.show(2)
df.count()

+--------+-----------+-----------+--------------+-----+
|coord_id|        lon|        lat|          date|value|
+--------+-----------+-----------+--------------+-----+
|      26|85.92405469|27.19137527|6/10/2011 0:00|  0.0|
|      26|85.92405469|27.19137527|6/10/2011 0:30|  0.0|
+--------+-----------+-----------+--------------+-----+
only showing top 2 rows



636240

In [50]:
# get num,ber of partitions
df_RailFall.rdd.getNumPartitions()

6

In [58]:
# Colesce dont shuffle data so falls under narrow transformation  
# and  generally used to reduce number of partitions
df_RailFall.coalesce(3).rdd.getNumPartitions()


3

In [57]:
#repartition shuffle data so falls under wide transformation
# and generally used to increase number of partitions but can also be used to reduce number of partitions
df_RailFall.repartition(30).rdd.getNumPartitions()

30

## Shared variables (Broadcast and Accumulators)